# Ruby 赋值机制

先看一个例子：

In [1]:
x = [1, 2, 3]
y = x
x[1] = 100
print y

[1, 100, 3]

改变变量`x`的值，变量`y`的值也随着改变，这与**Ruby**内部的赋值机制有关。

## 简单类型

先来看这一段代码在 **Ruby** 中的执行过程。

In [2]:
x = 500
y = x
y = 'foo'

x

500

**Ruby** 分配了一个 `Integer` 大小的内存 `pos1` 用来储存对象 `500` ，然后，**Ruby** 在命名空间中让变量 `x` 指向了这一块内存，注意，整数是不可变类型，所以这块内存的内容是不可变的。

| 内存 |命名空间|
| --- | --- |
|`pos1 : Integer(500)` (不可变)|          `x : pos1`         |

```ruby
y = x
```

**Ruby** 并没有使用新的内存来储存变量 `y` 的值，而是在命名空间中，让变量 `y` 与变量 `x` 指向了同一块内存空间。

|内存 | 命名空间|
| --- | --- |
|`pos1 : Integer(500)` (不可变)|`x : pos1`<br /> `y : pos1`|



```ruby
y = 'foo'
```

**Ruby** 此时分配一个 `String` 大小的内存 `pos2` 来储存对象 `foo` ，然后改变变量 `y` 所指的对象。

| 内存 | 命名空间|
| :--- | --- |
|`pos1 : Integer(500)` (不可变)<br /> `pos2 : String('foo')` (可变)|`x : pos1`<br />`y : pos2`|


对这一过程进行验证，可以使用 `object_id` 函数。

```ruby
x.object_id
```

返回变量 `x` 的内存地址。

In [4]:
x = 500
x.object_id

1001

In [5]:
y = x
y.object_id

1001

现在 `y` 指向另一块内存：

In [7]:
y = 'foo'
y.object_id

70181391938800

**Ruby** 会为每个出现的对象进行赋值，哪怕它们的值是一样的，例如：

In [8]:
x = 'foo'
x.object_id

70181391906460

In [9]:
y = 'foo'
y.object_id

70181391854320

In [13]:
x.equal?(y)

false

## 容器类型

现在来看另一段代码：

In [14]:
x = [500, 501, 502]
y = x
y[1] = 600
y = [700, 800]

[x, y]

[[500, 600, 502], [700, 800]]

```ruby
x = [500, 501, 502]
```

**Ruby** 为3个Integer分配内存 `pos1` ， `pos2` ， `pos3` （不可变），然后为列表分配一段内存 `pos4` ，它包含3个位置，分别指向这3个内存，最后再让变量 `x` 指向这个列表。

| 内存 | 命名空间 |
| :--- | --- |
|`pos1 : Integer(500)` (不可变) <br /> `pos2 : Integer(501)` (不可变) <br />`pos3 : Integer(502)` (不可变) <br /> `pos4 : Array(pos1, pos2, pos3)` (可变)|`x : pos4`|

```ruby
y = x
```

并没有创建新的对象，只需要将 `y` 指向 `pos4` 即可。

| 内存 | 命名空间 |
| :--- | --- |
|`pos1 : Integer(500)` (不可变) <br /> `pos2 : Integer(501)` (不可变) <br /> `pos3 : Integer(502)` (不可变) <br /> `pos4 : Array(pos1, pos2, pos3)` (可变)|`x : pos4`<br>`y : pos4`|


```ruby
y[1] = 600
```

原来 `y[1]` 这个位置指向的是 `pos2` ，由于不能修改 `pos2` 的值，所以首先为 `600` 分配新内存 `pos5` 。

再把 `y[1]` 指向的位置修改为 `pos5` 。此时，由于 `pos2` 位置的对象已经没有用了，**Ruby**会自动调用垃圾处理机制将它回收。

| 内存 | 命名空间 |
| :--- | --- |
|`pos1 : Integer(500)` (不可变) <br /> `pos2 :` 垃圾回收 <br /> `pos3 : Integer(502)` (不可变) <br /> `pos4 : Array(pos1, pos5, pos3)` (可变)<br />`pos5 : Integer(600)` (不可变) |`x : pos4`<br> `y : pos4`|


```ruby
y = [700, 800]
```

首先创建这个列表，然后将变量 `y` 指向它。

| 内存 | 命名空间 |
| :--- | ---: |
|`pos1 : Integer(500)` (不可变) <br/> `pos3 : Integer(502)` (不可变) <br />`pos4 : Integer(pos1, pos5, pos3)` (可变)<br />`pos5 : Integer(600)` (不可变) <br />`pos6 : Integer(700)` (不可变)<br>`pos7 : Integer(800)` (不可变)<br />`pos8 : Array(pos6, pos7)` (可变)|`x : pos4`<br /> `y : pos8`|

对这一过程进行验证：

In [17]:
x = [500, 501, 502]
puts x[0].object_id
puts x[1].object_id
puts x[2].object_id
puts x.object_id

1001
1003
1005
70181391531660


赋值，`y.object_id` 与 `x.object_id` 相同。

In [19]:
y = x
puts y.object_id

70181391531660


In [20]:
x.equal?(y)

true

修改 `y[1]` ，`y.object_id` 并不改变。

In [21]:
y[1] = 600
puts y.object_id

70181391531660


`x[1].object_id` 和 `y[1].object_id` 的值改变了。

In [22]:
puts x[1].object_id
puts y[1].object_id

1201
1201


更改 `y` 的值，`y.object_id` 的值改变

In [23]:
y = [700, 800]
puts y.object_id
puts x.object_id

70181393572940
70181391531660
